# Django & DRF – Backend Concepts Overview

This notebook contains an outline with code examples covering the most important backend concepts using Django and Django REST Framework.

---

## 📦 1. Models

### ➤ Model Creation & Fields  
```python
# models.py
from django.db import models

class Student(models.Model):
    name = models.CharField(max_length=100)
    age = models.IntegerField()
    created_at = models.DateTimeField(auto_now_add=True)    # set on record creation
    updated_at = models.DateTimeField(auto_now=True)          # updated on every save

    def __str__(self):
        return self.name
```

### ➤ Relationship Types

#### One-to-Many (ForeignKey)
```python
class Course(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField(blank=True)

    def __str__(self):
        return self.title

class Student(models.Model):
    # ... other fields ...
    # ForeignKey is used to create a one-to-many relationship
    course = models.ForeignKey(
        'Course', 
        on_delete=models.CASCADE, 
        related_name='students'
    )
    # 'Course' is a string to avoid circular imports. related_name=student is automati ally creating reverse relationship and course will have students attrbute in it.
```

#### One-to-One (OneToOneField)
```python
class Profile(models.Model):
    #
    student = models.OneToOneField(
        'Student', 
        on_delete=models.CASCADE, 
        related_name='profile'
    )
    bio = models.TextField(blank=True)
    avatar = models.ImageField(upload_to='avatars/', null=True, blank=True)
```

#### Many-to-Many (ManyToManyField)
```python
class Student(models.Model):
    # ... other fields ...
    courses = models.ManyToManyField(
        'Course', 
        related_name='enrolled_students'
    )

```

### ➤ Meta Class, Table Naming, Ordering, Constraints
```python
class Student(models.Model):
    name = models.CharField(max_length=100)
    age = models.IntegerField()
    # ... other fields ...

    # This approach ensures data validation happens at the database level, not just in your application code. 
    class Meta:
        ordering = ['name']
        db_table = 'student_table'
        verbose_name = 'Student'
        constraints = [
            models.CheckConstraint(check=models.Q(age__gte=0), name='age_non_negative')
        ]
```

### ➤ Custom Model Managers
```python
class ActiveStudentManager(models.Manager):
    def get_queryset(self):
        return super().get_queryset().filter(is_active=True)

class Student(models.Model):
    name = models.CharField(max_length=100)
    age = models.IntegerField()
    is_active = models.BooleanField(default=True)
    # ... other fields ...

    objects = models.Manager()         # default manager
    active_students = ActiveStudentManager()  # custom manager
```

---

## 🗃️ 2. Migrations & Database Handling

### ➤ makemigrations & migrate
```bash
# Terminal commands:
python manage.py makemigrations
python manage.py migrate
```

### ➤ Viewing SQL Generated by Django
```bash
python manage.py sqlmigrate app_name migration_number
```

### ➤ Resetting / Flushing the Database
```bash
python manage.py flush
```

### ➤ Fixtures & Seeding Data (Custom Commands)
Create a custom management command for seeding data:

```python
# management/commands/seed_db.py
# It will use seed.sql file to populate the database
from django.core.management.base import BaseCommand
from django.db import connection
from pathlib import Path
import os


class Command(BaseCommand):
    help = 'Populates the database with collections and products'

    def handle(self, *args, **options):
        print('Populating the database...')
        current_dir = os.path.dirname(__file__)
        file_path = os.path.join(current_dir, 'seed.sql')
        sql = Path(file_path).read_text()

        with connection.cursor() as cursor:
            cursor.execute(sql)
```

```bash
python manage.py seed_db    # Run the custom command
```

---

## 🖼️ 3. Admin Panel

### ➤ Registering Models & Customizing the Admin Interface
```python
# admin.py
from django.contrib import admin
from .models import Student, Course

class StudentAdmin(admin.ModelAdmin):
    list_display = ('name', 'age', 'course')    # Fields to display in the list view
    search_fields = ('name',)   # Add search functionality
    list_filter = ('course',)   # Add filters to the right sidebar
    fieldsets = (
        ('Basic Info', {'fields': ('name', 'age')}),
        ('Relationships', {'fields': ('course',)}),
    )   # Group fields while showing

class CourseAdmin(admin.ModelAdmin):
    list_display = ('title',)
    # Optionally, add inlines for related models

admin.site.register(Student, StudentAdmin)
admin.site.register(Course, CourseAdmin)
```

### ➤ Inline Models  
*(For showing related models on the same admin page, e.g., Student inline in Course admin)*
```python
class StudentInline(admin.TabularInline):
    model = Student
    extra = 1

class CourseAdmin(admin.ModelAdmin):
    inlines = [StudentInline]
```

---

## 🚦 4. Django Views

### ➤ Function-Based Views (FBVs)
```python
# views.py
from django.shortcuts import render
from .models import Student

def student_list(request):
    students = Student.objects.all()
    return render(request, 'students/student_list.html', {'students': students})
```

### ➤ Class-Based Views (CBVs) & Generic Views
```python
from django.views.generic import ListView

class StudentListView(ListView):
    model = Student
    template_name = 'students/student_list.html'
    context_object_name = 'students'
```

### ➤ Template Rendering
Make sure your templates are in the correct directory and use Django's templating language.

---

## 🔗 5. URLs & Routing

### ➤ URL Patterns and `path()`
```python
# urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('students/', views.student_list, name='student_list'),
    path('students/class/', views.StudentListView.as_view(), name='student_list_class'),
]
```

### ➤ Including App URLs
```python
# project-level urls.py
from django.urls import path, include

urlpatterns = [
    path('app/', include('myapp.urls')),
]
```

### ➤ Reverse URL Resolution
```python
from django.urls import reverse
url = reverse('student_list')
```

---

## 🧪 6. Testing in Django

### ➤ Unit Tests & TestCase
```python
# tests.py
from django.test import TestCase
from .models import Student, Course

class StudentModelTest(TestCase):
    def setUp(self):
        course = Course.objects.create(title='History', description='History course')
        Student.objects.create(name='Bob', age=22, course=course)

    def test_student_str(self):
        student = Student.objects.get(name='Bob')
        self.assertEqual(str(student), 'Bob')
```

### ➤ API Testing using APIClient (DRF)
```python
# tests_api.py
from rest_framework.test import APITestCase
from rest_framework import status
from django.urls import reverse

class StudentAPITest(APITestCase):
    def test_student_list_api(self):
        url = reverse('student_list')
        response = self.client.get(url)
        self.assertEqual(response.status_code, status.HTTP_200_OK)
```

---

# -----------------------------
# Django REST Framework (DRF)
# -----------------------------

## 🔧 7. DRF Basics

### ➤ Installation & Configuration
Ensure DRF is in your `INSTALLED_APPS` and configure any global settings:
```python
# settings.py (excerpt)
INSTALLED_APPS = [
    # default Django apps...
    'rest_framework',
    'myapp',  # your application
]

REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticatedOrReadOnly',
    ]
}
```

### ➤ Project & App Setup (Best Practices)
- Keep your API views, serializers, and URL routes organized in your app.

---

## 📥 8. Serializers

### ➤ Basic Serializers & ModelSerializers
```python
# serializers.Serializers
from rest_framework import serializers
from .models import Student, Course

class CourseSerializer(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    title = serializers.CharField(max_length=200)
    description = serializers.CharField()

    # Override create() and update() methods for custom behavior
    def create(self, validated_data):
        return Course.objects.create(**validated_data)

    def update(self, instance, validated_data):
        instance.title = validated_data.get('title', instance.title)
        instance.description = validated_data.get('description', instance.description)
        instance.save()
        return instance

# serializers.ModelSerializer
class StudentSerializer(serializers.ModelSerializer):
    custom_field = serializers.CharField(required=False)  # Add custom fields

    class Meta:
        model = Student
        fields = ['id', 'name', 'age', 'course', 'custom_field']

    # Add custom validation methods
    def validate_age(self, value):
        if value < 0:
            raise serializers.ValidationError("Age must be non-negative.")
        return value
    
```

### ➤ Nested Serializers
```python
# serializers.py
from rest_framework import serializers
from .models import Student, Course

class CourseSerializer(serializers.ModelSerializer):
    class Meta:
        model = Course
        fields = ['id', 'title', 'description']

class StudentSerializer(serializers.ModelSerializer):
    course = CourseSerializer()  # Nested serializer

    class Meta:
        model = Student
        fields = ['id', 'name', 'age', 'course']
    
    # Field-level validation example:
    def validate_age(self, value):
        if value < 0:
            raise serializers.ValidationError("Age must be non-negative.")
        return value
```
---

# **Django REST Framework (DRF) Views: Levels of Abstraction**
## **1️⃣ Function-Based Views: `@api_view` (Lowest Abstraction)**
🔹 `@api_view` is the simplest way to create API views in DRF.


### **Example:**
```python
from rest_framework.decorators import api_view
from rest_framework.response import Response

@api_view(['GET', 'POST'])
def my_view(request):
    if request.method == 'GET':
        return Response({"message": "GET request received"})
    elif request.method == 'POST':
        return Response({"message": "POST request received"})
```

---

## **2️⃣ Class-Based Views: `APIView` (More Control)**
🔹 `APIView` is the **class-based alternative** to `@api_view`.  
🔹 It provides more control than `@api_view`, allowing the use of **OOP principles**.  
✔ Handles authentication and permission checks.  
✔ Supports multiple HTTP methods using class-based methods (`get()`, `post()`, etc.).  

### **Example:**
```python
from rest_framework.views import APIView
from rest_framework.response import Response

class MyAPIView(APIView):
    def get(self, request):
        return Response({"message": "GET request received"})

    def post(self, request):
        return Response({"message": "POST request received"})
```

---

## **3️⃣ Generic Views: `GenericAPIView` (Base for Generic Views)**
🔹 `GenericAPIView` extends `APIView` but introduces **queryset, serializer_class, pagination, and filtering**.  
🔹 It **does not implement CRUD methods** but provides helper methods like `get_object()` and `get_queryset()`.
✔ **`queryset` and `serializer_class`** built-in for easy data handling.  
✔ Helper methods like `get_queryset()`, `get_object()`.  
✔ Provides the foundation for mixins and concrete views.

### **Example:**
```python
from rest_framework.generics import GenericAPIView
from rest_framework.response import Response
from myapp.models import User
from myapp.serializers import UserSerializer

class UserGenericAPIView(GenericAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer

    def get(self, request):
        users = self.get_queryset()
        serializer = self.get_serializer(users, many=True)
        return Response(serializer.data)
    
    def delete(self, request, pk):
        user = self.get_object()
        user.delete()
        return Response(status=204)
```

---

## **4️⃣ Mixins: Reusable View Logic**
🔹 **Mixins** provide reusable functionality for CRUD operations.  
🔹 They must be combined with `GenericAPIView`.  
🔹 They **don’t work alone** but help in reducing repetitive code.
✔ `CreateModelMixin` → Handles `POST` requests.  
✔ `ListModelMixin` → Handles `GET` (list) requests.  
✔ `RetrieveModelMixin` → Handles `GET` (single object).  
✔ `UpdateModelMixin` → Handles `PUT` and `PATCH` requests.  
✔ `DestroyModelMixin` → Handles `DELETE` requests.  

### **Example:**
```python
from rest_framework.generics import GenericAPIView
from rest_framework import mixins
from myapp.models import User
from myapp.serializers import UserSerializer

class UserListCreateView(mixins.ListModelMixin, mixins.CreateModelMixin, GenericAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer

    def get(self, request):
        return self.list(request)  # Calls ListModelMixin

    def post(self, request):
        return self.create(request)  # Calls CreateModelMixin
```

---

## **5️⃣ Concrete Generic Views (Pre-Built CRUD Views)**
🔹 **Concrete Views** are pre-built **CRUD views** that combine `GenericAPIView` and mixins.  
✔ `ListCreateAPIView` → Handles **GET (list) & POST (create)**.  
✔ `RetrieveUpdateDestroyAPIView` → Handles **GET (single), PUT, PATCH, DELETE**.  
✔ `RetrieveAPIView`, `UpdateAPIView`, `DestroyAPIView`, etc.  

### **Example:**
```python
from rest_framework.generics import ListCreateAPIView, RetrieveUpdateDestroyAPIView
from myapp.models import User
from myapp.serializers import UserSerializer

class UserListCreateView(ListCreateAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer

class UserRetrieveUpdateDestroyView(RetrieveUpdateDestroyAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer
```

---

## **6️⃣ ViewSets (Highest Abstraction)**
✔ Automatically handles **list, retrieve, create, update, delete**.  
✔ Works with **routers** for automatic URL routing.  
✔ Reduces boilerplate code.

### **Example:**
```python
from rest_framework.viewsets import ModelViewSet
from myapp.models import User
from myapp.serializers import UserSerializer

class UserViewSet(ModelViewSet):
    queryset = User.objects.all()
    serializer_class = UserSerializer
```
**Register with Router for Automatic URLs:**
```python
from rest_framework.routers import DefaultRouter
from myapp.views import UserViewSet

router = DefaultRouter()
router.register(r'users', UserViewSet)  # Generates /users/ endpoint
```
### ➤ SimpleRouter vs DefaultRouter
```python
# urls.py (DRF routing)
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from .drf_views import StudentViewSet

router = DefaultRouter()
router.register(r'students', StudentViewSet)

urlpatterns = [
    path('api/', include(router.urls)),
]
```

---

## **Summary: Levels of Abstraction**
| View Type | Provides |
|-----------|---------|
| `@api_view` | Simple function-based API views |
| `APIView` | Class-based views with methods (`get`, `post`, etc.) |
| `GenericAPIView` | Base class with queryset, serializer, and helpers | 
| `Mixins` + `GenericAPIView` | Reusable CRUD logic | 
| `Concrete Views` | Fully built CRUD views | 
| `ViewSets` | Auto-handles multiple actions | 

---

## **DRF Permissions & Authentication**

### **1️⃣ Permissions**
🔹 **Permissions** control access to views based on user authentication.  
🔹 **Permission classes** are defined in `REST_FRAMEWORK` settings.  
🔹 **Default permissions** include `AllowAny`, `IsAuthenticated`, `IsAdminUser`, etc.  
🔹 **Custom permissions** use `AllowAny`, `IsAuthenticated`, `IsAdminUser`, and extend them in a custom permission class.  

### **Example:**
```python
# Default permission classes
from rest_framework.permissions import IsAuthenticated
from rest_framework.views import APIView

class MyView(APIView):
    permission_classes = [IsAuthenticated]
```

---

```python
# Custom permission classes
from rest_framework.permissions import BasePermission

class CustomPermission(BasePermission):
    def has_permission(self, request, view):
        return (request.user and request.user.is_authenticated) or request.user.is_superuser
```

---

### **2️⃣ Authentication**
🔹 **Authentication** verifies the user's identity.  
🔹 **Authentication classes** are defined in `REST_FRAMEWORK` settings.  
🔹 **Default authentications** include `BasicAuthentication`, `SessionAuthentication`, etc.  

### **Example:**
```python
from rest_framework.authentication import BasicAuthentication
from rest_framework.views import APIView

class MyView(APIView):
    authentication_classes = [BasicAuthentication]
```

---

## **DRF Throttling, Filtering and Pagination**

### **1️⃣ Throttling**
🔹 **Throttling** limits the number of requests a user can make.  
🔹 **Throttling classes** are defined in `REST_FRAMEWORK` settings.  
🔹 **Default throttling** includes `AnonRateThrottle`, `UserRateThrottle`, etc.  

### **Example:**
```python
from rest_framework.throttling import UserRateThrottle
from rest_framework.views import APIView


class MyView(APIView):
    throttle_classes = [UserRateThrottle]
```

---

### **2️⃣ Filtering**
🔹 **Filtering** allows users to filter querysets using URL query parameters.  
🔹 **Filter backends** are defined in `REST_FRAMEWORK` settings.  
🔹 **Default filter backends** include `DjangoFilterBackend`, `SearchFilter`, etc.  

### **Example:**
```python
from rest_framework.filters import SearchFilter
from rest_framework.generics import ListAPIView
from myapp.models import User
from myapp.serializers import UserSerializer

class UserListView(ListAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer
    filter_backends = [SearchFilter]
    search_fields = ['name', 'email']
```

---

### **3️⃣ Pagination**
🔹 **Pagination** breaks up large responses into smaller, more manageable chunks.  
🔹 **Pagination classes** are defined in `REST_FRAMEWORK` settings.  
🔹 **Default pagination** includes `PageNumberPagination`, `LimitOffsetPagination`, etc.  

### **Example:**
```python
from rest_framework.pagination import PageNumberPagination
from rest_framework.generics import ListAPIView
from myapp.models import User
from myapp.serializers import UserSerializer

class UserListView(ListAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer
    pagination_class = PageNumberPagination
```

---